In [1]:
# -- IMPORTS START --
import pandas as pd
import glob
import re
import os
import sys
import pickle
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

from sklearn import tree, metrics
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay
from scipy.signal import butter, filtfilt, find_peaks
from sklearn.tree import DecisionTreeClassifier,export_graphviz
from sklearn.model_selection import train_test_split

import datetime
# -- IMPORTS END --

# enable zooming into graphs
%matplotlib notebook
plt.rcParams['figure.figsize'] = [9, 6] # width, height in inches

### Helper Functions:

**unix_to_date:**
- Converts Epoch(Unix) Time into traditional DateTime information

**starter:**
- applies all necessary cleaning to eliminate null instances, unnecessary columns, etc.
- returns working DataFrame with relevant information

**calc_magnitude**
- Takes individual(x,y,z) accelerometer information and combines it into a singular magnitude

**combine**
- Takes acceleration dataframe and combines it with gyroscope dataframe 

In [2]:
def unix_to_date(x):
    f = float(x)
    full = datetime.datetime.fromtimestamp(int(str(f)[:-5]))
    full = full.replace(microsecond=int(float(str(f)[-5:]))*1000)
    return full


In [4]:
def calc_magnitude(data):

    # Calculate magnitude  
    data['accel_mag'] = np.sqrt(data['x']**2 + data['y']**2 + data['z']**2) # absolute accel magnitude
    data['accel_mag'] = data['accel_mag'] - data['accel_mag'].mean() # detrend: "remove gravity"

    return data

In [5]:
def starter(filepath):
    df = pd.read_csv(filepath)
    df.dropna(inplace=True)
    df['Time'] = df['t_unix'].apply(unix_to_date)
    df = df.drop(['t_unix','t_Android','a'], axis=1)
    df = df.reindex(columns=['Time', 'x', 'y', 'z'])

    return df

In [6]:
def starter_g(filepath):
    df = pd.read_csv(filepath)
    df.dropna(inplace=True)
    df['Time'] = df['t_unix'].apply(unix_to_date)
    df.rename(columns={'x': 'x_gyro'},inplace=True)
    df.rename(columns={'y': 'y_gyro'},inplace=True)
    df.rename(columns={'z': 'z_gyro'},inplace=True)

    df = df.drop(['t_unix','t_Android','a'], axis=1)
    df = df.reindex(columns=['Time', 'x_gyro', 'y_gyro', 'z_gyro'])

    return df

In [7]:
def combine(acg,gyro):
    combined_df = pd.merge(acg,gyro,on="Time",how='left')
    combined_df.drop_duplicates(subset=acg, inplace=True)
    combined_df = combined_df.dropna()
    return combined_df



In [8]:
b_acg = starter("data/Backhand/WEAR_ACG.csv")
calc_magnitude(b_acg)
f_acg = starter("data/Forehand/WEAR_ACG.csv")
calc_magnitude(f_acg)
s_acg = starter("data/Serve/WEAR_ACG.csv")
calc_magnitude(s_acg)

b_gyro = starter_g("data/Backhand/WEAR_GYRO.csv")
f_gyro = starter_g("data/Forehand/WEAR_GYRO.csv")
s_gyro = starter_g("data/Serve/WEAR_GYRO.csv")

In [9]:
backhand_df = combine(b_acg,b_gyro)
forehand_df = combine(f_acg,f_gyro)
serve_df = combine(s_acg,s_gyro)

backhand_df.head()

,Time,x,y,z,accel_mag,x_gyro,y_gyro,z_gyro
0,2023-11-27 16:47:22.154,-19.558481,-13.191105,-11.038702,4.424051,0.434325,2.258522,8.095798
2,2023-11-27 16:47:22.228,-19.558481,-13.191105,-11.038702,4.424051,0.434325,2.258522,8.095798
10,2023-11-27 16:47:22.277,-19.558481,-13.191105,-11.038702,4.424051,0.434325,2.258522,8.095798
14,2023-11-27 16:47:22.310,-19.558481,-13.191105,-11.038702,4.424051,0.434325,2.258522,8.095798
18,2023-11-27 16:47:22.341,-19.558481,-13.191105,-11.038702,4.424051,0.434325,2.258522,8.095798
